# just use stored trained fies

In [ ]:
!pip install -q faiss-cpu tqdm scikit-learn ipywidgets matplotlib
!pip install -q torch torchvision
!pip install -q pandas pyarrow

   ━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/664.8 MB 107.7 MB/s eta 0:00:06ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in 
exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in 
wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", 
line 179, in resolve
    self.factory.preparer.prepare_linked_requirements_more(reqs)
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/operations/prepare.py", line 554, in 
prepare_linked_requirements_mo

In [ ]:
import faiss
import numpy as np
import pickle
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image, ImageOps
import os
import warnings

class JewelryRecommenderServing:
    def __init__(self,
                 vector_dimension: int = 1280,
                 index_path: str ="" ,
                 metadata_path: str ="" ):

        warnings.filterwarnings("ignore")

        # Load index and metadata
        self.index_path = index_path
        self.metadata_path = metadata_path
        self.index = None
        self.metadata = {}

        # Load model for feature extraction
        self.model = models.efficientnet_b0(weights='EfficientNet_B0_Weights.DEFAULT')
        self.model.eval()
        self.model = torch.nn.Sequential(*list(self.model.children())[:-1])

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = self.model.to(self.device)

        # Image transformation
        self.transform = transforms.Compose([
            transforms.Lambda(lambda img: ImageOps.exif_transpose(img)),
            transforms.Resize((640, 640)),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ])

        # Load the existing index and metadata
        self.load_index_and_metadata()

    def load_index_and_metadata(self) -> bool:
        """Load the pre-built FAISS index and metadata from files"""
        try:
            if os.path.exists(self.index_path) and os.path.exists(self.metadata_path):
                self.index = faiss.read_index(self.index_path)
                with open(self.metadata_path, "rb") as f:
                    self.metadata = pickle.load(f)
                print("Index and metadata loaded successfully.")
                return True
            else:
                print(f"Index file or metadata file not found at {self.index_path} or {self.metadata_path}")
                return False
        except Exception as e:
            print(f"Error loading index or metadata: {e}")
            return False

    def _extract_embedding(self, image_path: str) -> np.ndarray:
        """Extract embedding from an image using the pre-trained model"""
        try:
            with Image.open(image_path).convert('RGB') as img:
                img_tensor = self.transform(img).unsqueeze(0).to(self.device)
                with torch.no_grad():
                    embedding = self.model(img_tensor).squeeze().cpu().numpy()
                return embedding
        except Exception as e:
            print(f"Error processing image {image_path}: {str(e)}")
            return None

    def get_recommendations(self, query_image_path: str, num_recommendations: int = 5):
        """Get recommendations for a query image based on similarity"""
        if self.index is None:
            print("Index not loaded. Please check that the index path is correct.")
            return []

        query_embedding = self._extract_embedding(query_image_path)
        if query_embedding is None:
            return []

        # Perform the similarity search
        search_k = min(num_recommendations * 3, self.index.ntotal)
        distances, indices = self.index.search(query_embedding.reshape(1, -1), search_k)

        results = []
        seen_categories = set()

        for dist, idx in zip(distances[0], indices[0]):
            if idx != -1:
                metadata = self.metadata[idx]
                if metadata["full_path"] != query_image_path:
                    similarity_score = 1 / (1 + float(dist))
                    if metadata.get("category") not in seen_categories:
                        result = {
                            "metadata": metadata,
                            "distance": float(dist),
                            "similarity_score": similarity_score
                        }
                        results.append(result)
                        seen_categories.add(metadata.get("category"))

        results.sort(key=lambda x: x["similarity_score"], reverse=True)
        return results[:num_recommendations]


# Usage example:
def serve_recommendations(image_path, num_recommendations=5):
    # Initialize the recommender with paths to your saved files
    recommender = JewelryRecommenderServing(
        index_path= "/content/drive/MyDrive/chennai_TJ_project/model files/jewelry_index.idx",
        metadata_path=  "/content/drive/MyDrive/chennai_TJ_project/model files/jewelry_metadata.pkl"
    )

    # Get recommendations
    recommendations = recommender.get_recommendations(image_path, num_recommendations)

    return recommendations

In [ ]:
image_path="/content/811Aa-KZRnL._AC_UY1100_.jpg"
serve_recommendations(image_path)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 108MB/s] 


KeyboardInterrupt: 

# **load images data**


In [ ]:
# Block 1: Install required packages and import dependencies
!pip install -q gdown
import gdown
import os
import zipfile
from pathlib import Path


In [ ]:

# Block 2: Download and setup function
def setup_dataset_from_drive():
    """
    Downloads and sets up the jewelry dataset from Google Drive shared link
    Returns the path to the dataset
    """
    # Create base directory
    base_dir = "/content/extracted_jewellery_datar"
    os.makedirs(base_dir, exist_ok=True)

    # Extract file ID from the Google Drive link
    file_id = "1z445s15uuZUysdpyOYjIbWcV0CQrO5fs"

    # Construct the direct download URL
    url = f"https://drive.google.com/uc?id={file_id}"

    # Download location
    # zip_path = os.path.join(base_dir, "jewelry_dataset.zip")
    zip_path = "/content/drive/MyDrive/chennai_TJ_project/datafiles/all images extracted.zip"
    print("Downloading dataset from Google Drive...")
    try:
        # Download the file
        # if you got zip file upload it and comment the below line to avoid downloading again👋
        # gdown.download(url, zip_path, quiet=False)

        print("\nExtracting files...")
        # Extract the zip file
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(base_dir)

        # Remove the zip file to save space
        os.remove(zip_path)

        # Verify the dataset path exists
        dataset_path = "/content/extracted_jewellery_datar/all images extracted"
        if os.path.exists(dataset_path):
            print(f"\nDataset successfully downloaded and extracted to: {dataset_path}")
            # Count images
            image_count = len(list(Path(dataset_path).rglob("*.[jJ][pP][gG]")))
            print(f"Found {image_count} images in the dataset")
            return dataset_path
        else:
            print(f"\nError: Expected dataset path not found: {dataset_path}")
            return None

    except Exception as e:
        print(f"\nError downloading or extracting dataset: {e}")
        return None

# Block 3: Execute setup
if __name__ == "__main__":
    dataset_path = setup_dataset_from_drive()


Extracting files...

Dataset successfully downloaded and extracted to: /content/extracted_jewellery_datar/all images extracted
Found 126985 images in the dataset


# **DEPENDENCIES INSTALLATION**

Checking installed packages:
✓ faiss-cpu (version 1.10.0)
✓ tqdm (version 4.67.1)
✓ scikit-learn (version 1.6.1)
✓ ipywidgets (version 7.7.1)
✓ matplotlib (version 3.10.0)
✓ torch (version 2.5.1+cu124)
✓ torchvision (version 0.20.1+cu124)
✓ pandas (version 2.2.2)
✓ pyarrow (version 17.0.0)
✓ Pillow (version 11.1.0)
✓ gdown (version 5.2.0)
✓ concurrent-log-handler (version 0.9.25)
✓ plotly (version 5.24.1)
✓ notebook (version 6.5.5)


In [ ]:
# Block 1: Install required packages
!pip install -q faiss-cpu tqdm scikit-learn ipywidgets matplotlib
!pip install -q torch torchvision
!pip install -q pandas pyarrow
!pip install -q Pillow
!pip install -q gdown
!pip install -q concurrent-log-handler
!pip install -q plotly  # for enhanced visualizations
!pip install -q notebook  # ensure notebook compatibility
!pip install -q ipywidgets
!pip install -q typing-extensions  # for enhanced type hints

# Block 2: Verify installations
import pkg_resources
import sys

def verify_installations():
    required_packages = [
        'faiss-cpu',
        'tqdm',
        'scikit-learn',
        'ipywidgets',
        'matplotlib',
        'torch',
        'torchvision',
        'pandas',
        'pyarrow',
        'Pillow',
        'gdown',
        'concurrent-log-handler',
        'plotly',
        'notebook'
    ]

    print("Checking installed packages:")
    for package in required_packages:
        try:
            version = pkg_resources.get_distribution(package).version
            print(f"✓ {package} (version {version})")
        except pkg_resources.DistributionNotFound:
            print(f"✗ {package} not found!")

# Run verification
if __name__ == "__main__":
    verify_installations()

    # Print Python version
    print(f"\nPython version: {sys.version}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.5 MB/s eta 0:00:00


<ipython-input-3-da9a27fa28a1>:14: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


Checking installed packages:
✓ faiss-cpu (version 1.10.0)
✓ tqdm (version 4.67.1)
✓ scikit-learn (version 1.6.1)
✓ ipywidgets (version 7.7.1)
✓ matplotlib (version 3.10.0)
✓ torch (version 2.5.1+cu124)
✓ torchvision (version 0.20.1+cu124)
✓ pandas (version 2.2.2)
✓ pyarrow (version 18.1.0)
✓ Pillow (version 11.1.0)
✓ gdown (version 5.2.0)
✓ concurrent-log-handler (version 0.9.25)
✓ plotly (version 5.24.1)
✓ notebook (version 6.5.5)

Python version: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]


In [ ]:
# Block 1: Import Dependencies
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image, ImageOps
import faiss
import numpy as np
from pathlib import Path
import pickle
import logging
from tqdm.auto import tqdm
import os
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from google.colab import files
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import warnings
warnings.filterwarnings("ignore")

# Suppress verbose logging
logging.getLogger('matplotlib').setLevel(logging.WARNING)
logging.getLogger('PIL').setLevel(logging.WARNING)

from dataclasses import dataclass
from typing import Dict, Tuple, List, Optional
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score


# Advanced clustering


In [ ]:

@dataclass
class ClusterMetrics:
    n_clusters: int
    silhouette: float
    davies_bouldin: float
    cluster_sizes: Dict[int, int]
    inertia: float

class EnhancedJewelryClusterer:
    def __init__(self,
                 min_clusters: int = 10,
                 max_clusters: int = 50,
                 random_state: int = 42):
        self.min_clusters = min_clusters
        self.max_clusters = max_clusters
        self.random_state = random_state
        self.best_model = None
        self.cluster_centers_ = None
        self.scaler = StandardScaler()

    def analyze_jewelry_types(self, metadata: List[Dict]) -> Dict[str, int]:
        """Analyze distribution of jewelry types in dataset"""
        type_counts = {}
        for item in metadata:
            j_type = item.get('jewelry_type', 'unknown')
            type_counts[j_type] = type_counts.get(j_type, 0) + 1
        return type_counts

    def adjust_clusters_by_complexity(self, n_samples: int) -> Tuple[int, int]:
        """Adjust cluster range based on dataset size and complexity"""
        # Base calculation
        suggested_min = max(10, n_samples // 1000)
        suggested_max = min(50, n_samples // 100)

        # Ensure reasonable bounds
        final_min = max(self.min_clusters, suggested_min)
        final_max = min(self.max_clusters, suggested_max)
        final_min = min(final_min, final_max)  # Added line to ensure min_k <= max_k

        return final_min, final_max

    def evaluate_clustering(self,
                          embeddings: np.ndarray,
                          n_clusters: int) -> ClusterMetrics:
        """Evaluate clustering for a specific number of clusters"""
        kmeans = KMeans(n_clusters=n_clusters,
                       random_state=self.random_state,
                       n_init='auto')

        # Fit and predict
        labels = kmeans.fit_predict(embeddings)

        # Calculate metrics
        sil_score = silhouette_score(embeddings, labels)
        db_score = davies_bouldin_score(embeddings, labels)

        # Get cluster sizes
        unique, counts = np.unique(labels, return_counts=True)
        cluster_sizes = dict(zip(unique, counts))

        return ClusterMetrics(
            n_clusters=n_clusters,
            silhouette=sil_score,
            davies_bouldin=db_score,
            cluster_sizes=cluster_sizes,
            inertia=kmeans.inertia_
        )

    def find_optimal_clusters(self,
                            embeddings: np.ndarray,
                            metadata: Optional[List[Dict]] = None) -> Dict:
        """Find optimal number of clusters using multiple metrics"""
        print("Starting clustering analysis...")

        # Scale the embeddings
        scaled_embeddings = self.scaler.fit_transform(embeddings)

        # Adjust cluster range based on dataset size
        min_k, max_k = self.adjust_clusters_by_complexity(len(embeddings))
        print(f"Analyzing cluster range: {min_k} to {max_k}")

        # Analyze jewelry types if metadata available
        if metadata:
            type_distribution = self.analyze_jewelry_types(metadata)
            print("\nJewelry Type Distribution:")
            for j_type, count in type_distribution.items():
                print(f"{j_type}: {count} items ({count/len(metadata)*100:.1f}%)")

        # Evaluate different cluster counts
        metrics_list = []
        for k in tqdm(range(min_k, max_k + 1, 2), desc="Evaluating clusters"):
            metrics = self.evaluate_clustering(scaled_embeddings, k)
            metrics_list.append(metrics)

        # Find best configuration using combined metric
        best_metrics = max(metrics_list,
                         key=lambda x: x.silhouette - x.davies_bouldin * 0.5)

        # Fit final model with optimal clusters
        final_model = KMeans(n_clusters=best_metrics.n_clusters,
                           random_state=self.random_state,
                           n_init='auto')
        final_labels = final_model.fit_predict(scaled_embeddings)

        # Store best model and cluster centers
        self.best_model = final_model
        self.cluster_centers_ = final_model.cluster_centers_

        # Prepare detailed report
        report = {
            'optimal_clusters': best_metrics.n_clusters,
            'silhouette_score': best_metrics.silhouette,
            'davies_bouldin_score': best_metrics.davies_bouldin,
            'cluster_distribution': best_metrics.cluster_sizes,
            'cluster_labels': final_labels,
            'scaled_embeddings': scaled_embeddings
        }

        # Print summary
        print("\nClustering Analysis Results:")
        print(f"Optimal number of clusters: {report['optimal_clusters']}")
        print(f"Silhouette Score: {report['silhouette_score']:.3f}")
        print(f"Davies-Bouldin Score: {report['davies_bouldin_score']:.3f}")

        print("\nCluster Size Distribution:")
        for cluster, size in report['cluster_distribution'].items():
            percentage = (size / len(embeddings)) * 100
            print(f"Cluster {cluster}: {size} items ({percentage:.1f}%)")

        return report

    def predict(self, embeddings: np.ndarray) -> np.ndarray:
        """Predict clusters for new embeddings"""
        if self.best_model is None:
            raise ValueError("Model not fitted. Run find_optimal_clusters first.")

        scaled_embeddings = self.scaler.transform(embeddings)
        return self.best_model.predict(scaled_embeddings)

# Integration with JewelryRecommender
def enhanced_auto_categorize_images(self, embeddings_array: np.ndarray) -> np.ndarray:
    """
    Enhanced auto-categorization method for JewelryRecommender class
    """
    clusterer = EnhancedJewelryClusterer()
    clustering_report = clusterer.find_optimal_clusters(
        embeddings_array,
        metadata=list(self.metadata.values()) if hasattr(self, 'metadata') else None
    )

    # Store clustering information in metadata
    self.metadata['clustering_info'] = {
        'optimal_clusters': clustering_report['optimal_clusters'],
        'silhouette_score': clustering_report['silhouette_score'],
        'davies_bouldin_score': clustering_report['davies_bouldin_score'],
        'cluster_centers': clusterer.cluster_centers_.tolist()
    }

    return clustering_report['cluster_labels']

# **Main call**

In [ ]:

# Block 2: JewelryRecommender Class Definition
class JewelryRecommender:
    def __init__(self,
                dataset_path: str = "",
                vector_dimension: int = 1280,
                index_path: str = "/content/drive/MyDrive/chennai_TJ_project/model files/jewelry_index.idx",
                metadata_path: str = "/content/drive/MyDrive/chennai_TJ_project/model files/jewelry_metadata.pkl",
                vectors_path: str = "/content/drive/MyDrive/chennai_TJ_project/model files/jewelry_vectors_1281.parquet",
                num_clusters: Optional[int] = None):  # Changed from fixed 10

        logging.basicConfig(level=logging.ERROR)
        self.logger = logging.getLogger(__name__)

        self.dataset_path = dataset_path
        self.num_clusters = num_clusters
        self.index_path = index_path
        self.metadata_path = metadata_path
        self.vectors_path = vectors_path

        # Initialize FAISS index
        self.index = faiss.IndexIVFFlat(
            faiss.IndexFlatL2(vector_dimension),
            vector_dimension,
            min(100, max(10, int(vector_dimension * 0.1))),
            faiss.METRIC_L2
        )

        # Load model
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=UserWarning)
            self.model = models.efficientnet_b0(weights='EfficientNet_B0_Weights.DEFAULT')
            self.model.eval()
            self.model = torch.nn.Sequential(*list(self.model.children())[:-1])

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = self.model.to(self.device)

        # Image transformation
        self.transform = transforms.Compose([
            transforms.Lambda(lambda img: ImageOps.exif_transpose(img)),
            transforms.Resize((640, 640)),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ])

        self.metadata = {}

    # Block 3: Feature Extraction Methods
    def _extract_embedding(self, image_path: str) -> Optional[np.ndarray]:
        try:
            with Image.open(image_path).convert('RGB') as img:
                img_tensor = self.transform(img).unsqueeze(0).to(self.device)
                with torch.no_grad():
                    embedding = self.model(img_tensor).squeeze().cpu().numpy()
                return embedding
        except Exception as e:
            print(f"Error processing image {image_path}: {str(e)}")
            return None


    # Integration with JewelryRecommender
    def enhanced_auto_categorize_images(self, embeddings_array: np.ndarray) -> np.ndarray:
        """
        Enhanced auto-categorization method for JewelryRecommender class
        """
        clusterer = EnhancedJewelryClusterer()
        clustering_report = clusterer.find_optimal_clusters(
            embeddings_array,
            metadata=list(self.metadata.values()) if hasattr(self, 'metadata') else None
        )

        # Store clustering information in metadata
        self.metadata['clustering_info'] = {
            'optimal_clusters': clustering_report['optimal_clusters'],
            'silhouette_score': clustering_report['silhouette_score'],
            'davies_bouldin_score': clustering_report['davies_bouldin_score'],
            'cluster_centers': clusterer.cluster_centers_.tolist()
        }

        return clustering_report['cluster_labels']
    # Block 4: Index and Storage Management Methods
    def save_vectors_to_parquet(self, embeddings_array: np.ndarray, image_paths: List[str]):
        """Save vectors to parquet file with columns for each dimension"""
        # Create column names for each dimension
        dim_cols = [f'dim_{i}' for i in range(embeddings_array.shape[1])]

        # Create DataFrame with embeddings
        df = pd.DataFrame(embeddings_array, columns=dim_cols)
        df['image_path'] = image_paths

        # Save to parquet
        df.to_parquet(self.vectors_path, index=False)
        print(f"Vectors saved to {self.vectors_path}")

    def load_vectors_from_parquet(self) -> tuple[np.ndarray, List[str]]:
        """Load vectors from parquet file"""
        if not os.path.exists(self.vectors_path):
            return None, None

        df = pd.read_parquet(self.vectors_path)
        image_paths = df['image_path'].tolist()
        dim_cols = [col for col in df.columns if col.startswith('dim_')]
        embeddings_array = df[dim_cols].values

        return embeddings_array, image_paths

    def load_index_and_metadata(self) -> bool:
        try:
            if os.path.exists(self.index_path) and os.path.exists(self.metadata_path):
                self.index = faiss.read_index(self.index_path)
                with open(self.metadata_path, "rb") as f:
                    self.metadata = pickle.load(f)
                print("Index and metadata loaded successfully.")
                return True
        except Exception as e:
            print(f"Error loading index or metadata: {e}")
        return False

    def _save_index_and_metadata(self):
        try:
            os.makedirs(os.path.dirname(self.index_path) or '.', exist_ok=True)
            faiss.write_index(self.index, self.index_path)
            with open(self.metadata_path, "wb") as f:
                pickle.dump(self.metadata, f)
            print("Index and metadata saved successfully.")
        except Exception as e:
            print(f"Error saving index or metadata: {e}")

    # Block 5: Index Building Method
    def build_index(self):
        all_images = list(Path(self.dataset_path).rglob("*.[jJ][pP][gG]"))
        total_images = len(all_images)
        print(f"Found {total_images} images in the dataset")

        if total_images == 0:
            print("No images found in the dataset. Exiting index building.")
            return

        embeddings_list = []
        metadata_list = []
        image_paths = []

        for path in tqdm(all_images):
            embedding = self._extract_embedding(str(path))
            if embedding is not None:
                embeddings_list.append(embedding)
                image_paths.append(str(path))
                metadata = {
                    "full_path": str(path),
                    "filename": path.name
                }
                metadata_list.append(metadata)

        if embeddings_list:
            embeddings_array = np.vstack(embeddings_list)

            # Save vectors to parquet
            self.save_vectors_to_parquet(embeddings_array, image_paths)

            # Auto-categorize images
            categories = self.enhanced_auto_categorize_images(embeddings_array)
            for meta, cat in zip(metadata_list, categories):
                meta["category"] = f"Category_{cat}"

            print("Training the index...")
            self.index.train(embeddings_array)

            print("Adding images to the index...")
            ids = np.arange(len(embeddings_list))
            self.index.add_with_ids(embeddings_array, ids)
            self.metadata = {i: meta for i, meta in enumerate(metadata_list)}

            self._save_index_and_metadata()
            print(f"Successfully indexed {len(embeddings_list)} images")

    # Block 6: Recommendation Methods
    def get_recommendations(self, query_image_path: str, num_recommendations: int = 5) -> List[Dict]:
        query_embedding = self._extract_embedding(query_image_path)
        if query_embedding is None:
            return []

        search_k = min(num_recommendations * 3, self.index.ntotal)
        # print('search_k vallue:',search_k)
        distances, indices = self.index.search(query_embedding.reshape(1, -1), search_k)

        results = []
        seen_categories = set()

        for dist, idx in zip(distances[0], indices[0]):
            if idx != -1:
                metadata = self.metadata[idx]
                if metadata["full_path"] != query_image_path:
                    similarity_score = 1 / (1 + float(dist))
                    if metadata.get("category") not in seen_categories:
                        result = {
                            "metadata": metadata,
                            "distance": float(dist),
                            "similarity_score": similarity_score
                        }
                        results.append(result)
                        seen_categories.add(metadata.get("category"))

        results.sort(key=lambda x: x["similarity_score"], reverse=True)
        return results[:num_recommendations]

    def display_recommendations(self, query_image_path: str, num_recommendations: int = 5):
        recommendations = self.get_recommendations(query_image_path, num_recommendations)

        if not recommendations:
            print("No recommendations found.")
            return

        plt.figure(figsize=(20, 5))

        plt.subplot(1, num_recommendations + 1, 1)
        query_img = Image.open(query_image_path).convert('RGB')
        plt.imshow(query_img)
        plt.title('Query Image', fontsize=10)
        plt.axis('off')

        for idx, result in enumerate(recommendations, 2):
            plt.subplot(1, num_recommendations + 1, idx)
            img_path = result['metadata']['full_path']
            img = Image.open(img_path).convert('RGB')
            plt.imshow(img)

            similarity = result['similarity_score']
            plt.title(f"Match {idx-1}\nSimilarity: {similarity:.3f}\nCategory: {result['metadata'].get('category', 'N/A')}",
                      fontsize=8)
            plt.axis('off')

        plt.tight_layout()
        plt.show()

        print("\nDetailed Recommendations:")
        for idx, result in enumerate(recommendations, 1):
            print(f"\n{idx}. Category: {result['metadata'].get('category', 'N/A')}")
            print(f"   Similarity Score: {result['similarity_score']:.3f}")
            print(f"   File: {result['metadata']['filename']}")

# Block 7: Interface Creation
def create_colab_interface(recommender):
    output_area = widgets.Output()

    def on_upload_button_clicked(b):
        with output_area:
            clear_output()
            print("Upload an image to get recommendations")
            uploaded = files.upload()

            if uploaded:
                filename = list(uploaded.keys())[0]
                try:
                    recommender.display_recommendations(filename)
                except Exception as e:
                    print(f"Error processing image: {e}")

    def on_sample_button_clicked(b):
        with output_area:
            clear_output()
            dataset_images = list(Path(recommender.dataset_path).rglob("*.[jJ][pP][gG]"))

            if dataset_images:
                sample_image = str(dataset_images[0])
                print(f"Using sample image: {sample_image}")
                recommender.display_recommendations(sample_image)
            else:
                print("No sample images found in the dataset.")

    upload_button = widgets.Button(
        description='Upload Image',
        button_style='primary',
        layout=widgets.Layout(width='200px')
    )

    sample_button = widgets.Button(
        description='Use Sample Image',
        button_style='success',
        layout=widgets.Layout(width='200px')
    )

    upload_button.on_click(on_upload_button_clicked)
    sample_button.on_click(on_sample_button_clicked)

    box = widgets.VBox([
        widgets.HTML("<h2>Jewelry Recommendation System</h2>"),
        widgets.HBox([upload_button, sample_button]),
        output_area
    ])

    display(box)

# Block 8: Main Execution
def main():
    dataset_path = "/content/extracted_jewellery_datar/all images extracted"

    # Initialize recommender
    recommender = JewelryRecommender(dataset_path=dataset_path)

    # Build or load index
    if not recommender.load_index_and_metadata():
        print("Building new index...")
        recommender.build_index()

    # Create and display interactive interface
    create_colab_interface(recommender)

if __name__ == "__main__":
    main()

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 40.5MB/s]


Index and metadata loaded successfully.
